In [3]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED, DATABASE_DIR
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file, TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [4]:
# cluster
# cluster_netgen
# compression
# expansion
# explosion
# grid
# implosion
# linearprojection
# rotation
# uniform_portgen

In [5]:
N = 30
instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json"
)

number_of_instances = N // 5
train_instances = InstanceList()
test_instances = InstanceList()
for i in range(10):
    test_instances.append(instances[i * 50])
for i in range(1, 6):
    train_instances.extend(instances[i * 50 + 1 : i * 50 + number_of_instances + 1])

In [6]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=2)
portfolio

Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]

In [ ]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    max_iter=75,
    calculate_features=True,
    estimator=None,
)

last_model_iter = 0

estimator = None
db = DB()
estimator_pct = 0.5
for _ in aac.configure_iter():
    if aac.iter > 25 and aac.iter - last_model_iter >= 5:
        X, y = get_model_training_data(db)
        estimator = Estimator1(
            max_cost=TSP_LKH_Solver.MAX_COST, estimator_pct=estimator_pct
        )
        estimator.fit(X, y)
        estimator.log()
        last_model_iter = aac.iter
        aac.update(estimator=estimator)

In [10]:
# db = DB()
db = DB("database/run-plain-70-925205.db")

In [19]:
X, y = get_model_training_data(db)
estimator_pct = 0.5
estimator = Estimator1(
    max_cost=TSP_LKH_Solver.MAX_COST, estimator_pct=estimator_pct
)
estimator.fit(X, y)
estimator.log()

[2025-01-01 17:05:52] DEBUG     Estimator1(fitted=True, score=-1.0267551249279099, estimator_pct=0.5, training_data_shape=(10500, 137), non_timeout=(1421, 137))


In [11]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,...,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,822075595471039179,TSP/CEPS_benchmark/cluster_netgen/01.tsp,9080004.0,0.011018,1.411569,1.481377,3.129632,0.897300,3.118615,0.605720,...,0.055556,0.5,0.5,0.5,0.944444,0.85250,0.29625,0.32,324.998924,0.68
1,411598327185181776,TSP/CEPS_benchmark/cluster_netgen/02.tsp,11020488.0,0.006047,1.536236,1.561506,3.138415,0.923695,3.132368,0.591541,...,0.083333,0.5,0.5,0.5,0.916667,0.86750,0.26750,0.20,428.028849,0.43
2,56462699908304675,TSP/CEPS_benchmark/cluster_netgen/03.tsp,9746976.0,0.005927,1.392946,1.468175,3.140087,0.912639,3.134160,0.621614,...,0.166667,0.5,0.5,0.5,0.833333,0.87125,0.25875,0.23,395.037079,0.43
3,1135796868948507057,TSP/CEPS_benchmark/cluster_netgen/04.tsp,8918339.0,0.003895,1.407044,1.487371,3.135897,0.929052,3.132002,0.624627,...,0.250000,0.5,0.5,0.5,0.750000,0.90000,0.20250,0.19,418.453921,0.44
4,900769072242699328,TSP/CEPS_benchmark/cluster_netgen/05.tsp,10525131.0,0.004843,1.532593,1.541054,3.139785,0.893347,3.134942,0.579699,...,0.166667,0.5,0.5,0.5,0.850000,0.86125,0.28000,0.24,391.845650,0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1765803814609198146,TSP/CEPS_benchmark/grid/00.tsp,21258773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,730618644385552802,TSP/CEPS_benchmark/implosion/00.tsp,20437744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,396111139638522607,TSP/CEPS_benchmark/linearprojection/00.tsp,13972705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,310544774750200091,TSP/CEPS_benchmark/rotation/00.tsp,16119107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.00,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,473831520656465062,0.25,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.50,...,0.470388,0.888889,1.00,0.6,0.2,0.244898,0.0,1.0,3.0,1.0
2,1724561038518932584,0.30,0.0,1.0,2.0,0.6,0.0,1.0,1.0,0.50,...,0.192348,0.777778,0.50,0.4,0.4,0.571429,0.0,1.0,5.0,0.0
3,388564707472054326,0.65,1.0,0.0,3.0,0.8,0.0,0.0,1.0,1.00,...,0.547346,0.222222,0.50,0.8,0.0,0.377551,1.0,0.0,2.0,0.0
4,2169120517035206132,0.35,1.0,1.0,1.0,0.7,0.0,0.0,0.0,0.00,...,0.197133,0.333333,0.25,1.0,1.0,0.510204,1.0,1.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,714327894705390055,0.20,1.0,1.0,0.0,0.5,0.0,1.0,1.0,0.00,...,0.044507,0.000000,0.25,0.4,0.4,0.132653,1.0,1.0,1.0,1.0
146,1135984693804820881,0.45,0.0,1.0,2.0,0.5,0.0,1.0,0.0,0.50,...,0.922970,0.333333,0.00,0.8,0.2,0.020408,1.0,1.0,4.0,0.0
147,432818753181671919,0.40,0.0,0.0,3.0,0.5,0.0,1.0,0.0,1.00,...,0.282639,0.000000,0.50,1.0,0.6,0.000000,1.0,1.0,4.0,1.0
148,1604016302140147593,0.15,0.0,1.0,2.0,0.5,0.0,1.0,0.0,0.25,...,0.968107,0.000000,0.00,0.8,0.4,0.020408,0.0,1.0,4.0,0.0


In [13]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,config;aac_iter=1_473831520656465062_822075595...,config;aac_iter=1,473831520656465062,822075595471039179,100.00,10.000000,0,0,0
1,config;aac_iter=1_52980316141715150_8220755954...,config;aac_iter=1,52980316141715150,822075595471039179,100.00,21.905660,0,0,1
2,config;aac_iter=1_52980316141715150_4115983271...,config;aac_iter=1,52980316141715150,411598327185181776,100.00,21.180797,0,0,0
3,config;aac_iter=1_473831520656465062_411598327...,config;aac_iter=1,473831520656465062,411598327185181776,100.00,10.000000,0,0,0
4,config;aac_iter=1_52980316141715150_5646269990...,config;aac_iter=1,52980316141715150,56462699908304675,100.00,20.414095,0,0,1
...,...,...,...,...,...,...,...,...,...
10695,test9_729675616749885658_730618644385552802,test9,729675616749885658,730618644385552802,100.00,10.000000,0,0,0
10696,test9_729675616749885658_1239352391178630764,test9,729675616749885658,1239352391178630764,7.38,7.380000,0,0,0
10697,test9_729675616749885658_396111139638522607,test9,729675616749885658,396111139638522607,100.00,10.000000,0,0,0
10698,test9_729675616749885658_310544774750200091,test9,729675616749885658,310544774750200091,100.00,10.000000,0,0,0


In [13]:
results["error"].value_counts()

0    8710
1    1990
Name: error, dtype: int64